In [55]:
def stitch_finder(text):
    """
    Detect substrings using a sliding window.

    :param text: The input string to search.
    :param targets: A list of target substrings to detect.
    :return: A list of lists, each containing detected substrings with their numbers for each line.
    """
    # Normalize the text and targets for case-insensitive search
    text = text.lower()
    text_result = []
    targets = ["sc", "ch", "dc", "hdc", "tc", "mr", "inc", "dec", "blo", "flo", "slst", "x"]

    # Split the text into lines
    lines = text.splitlines()

    # Initialize results
    all_results = []

    # Sliding window approach
    window_size = max(len(target) for target in targets)  # Max length of target substrings

    repeat = False
    toadd = []
    postorepeat = []
    for line in lines:
        wordcount = 0
        if line:
            line_results = []
            line += '   '
            line = line.replace('sl st', 'slst')
            

            for i in range(len(line) - window_size + 1):
                if line[i] == ' ' and i < len(line) -1 and line[i+1] != ' ':
                    wordcount += 1
                    continue

                # Extract the current window
                window = line[i:i + window_size]
                if window.startswith('[') or window.startswith('('):
                    repeat = True
                    
                elif window.startswith(']') or window.startswith(')'):
                    if toadd:
                        line_results.append(toadd)
                    toadd = []
                    repeat = False
                #print(window)
                # Check for any target in the window
                for target in targets:

                    


                    if window.startswith(target):
                        # Ensure the characters before and after are not letters
                        before = line[i - 1] if i > 0 else " "
                        after = line[i + len(target)] if i + len(target) < len(line) else " "

                        if not before.isalpha() and not after.isalpha():
                        # Extract the number before the target
                            if target == "x":
                                num_start = i + 1
                                while num_start < len(line) and (line[num_start].isdigit() or line[num_start] == ' '):
                                    num_start += 1

                                number = int(line[i+1:num_start].strip())
                                #print(number)
                                torepeat = line_results[-1]
                                line_results.pop()
                                count = torepeat[0][1]
                                for j in range(number):
                                    
                                    for item, oldcount in torepeat:
                                        line_results.append((item, count))
                                        count += 1
                                        if j != number-1:
                                            line = line[:i] + item + ' ' + line[i:]
                                            i += len(item) + 1

                                #result = f"{target}{number}".strip()
                                #line_results.append(result)
                                num_start = i + 1
                                while num_start < len(line) and (line[num_start].isdigit() or line[num_start] == ' '):
                                    num_start += 1
                                
                                line = line[:i] + line[num_start:]
                                print(line[i], line)
                                print(torepeat, line_results)

                                break

                            else:
                                num_start = i - 1
                                while num_start >= 0 and (line[num_start].isdigit() or line[num_start] == ' '):
                                    num_start -= 1

                                
                                if num_start < i - 2:
                                    number = line[num_start + 1:i].strip() 
                                    line = line[:i-1] + line[i:]
                                    wordcount -= 1
                                else:
                                    number = ""

                                # Exclude matches where the characters before (excluding spaces) are "th", "nd", or "st"
                                context_start = max(0, num_start - 2)
                                context = line[context_start:num_start + 1].strip()

                                if context not in ["th", "nd", "st"]:
                                    
                                    result = f"{number}{target}".strip()
                                    if repeat:
                                        toadd.append((result, wordcount))
                                    else:
                                       
                                        line_results.append((result, wordcount))
                                        print("window", window, line, line[i], i)
                                        #print(line_results)
                                break  # Move to the next window after finding a match
            if line_results:
                all_results.append(line_results)
            
            line = line.replace('\t', ' ')[:-3]
            
            text_result.append(line)
        
    return (text_result, all_results)


text = """
R1\t        6 SC in a MR

R2\t        [INC] x 6  (12)

R3\t        [SC, INC] x 6 (18)

R4\t        16 SC, [SL ST, CH1, DC, CH1, SL ST] in FLO of one stitch, 1 SC

If you need more help with round 4, please visit the video tutorial at timestamp: 10:52

R5\t        2 SC, [SL ST, CH1, DC CH1, SL ST] in  FLO of one stitch, 15 SC (the 14th SC should be made in the BLO of R3)

R6\t        7 SC (the third SC should be made in the BLO of R4), 4-DcBo , 3 SC,  4-DcBo, 6 SC (18)

R7-9\t18 SC (18)   3 Rounds

R10\t        8 SC, 4-DcBo, 3 SC, 4-DcBo, 5 SC (18)
"""

print(stitch_finder(text))

window sc i r1	        6sc in a mr    c 13
window mr   r1	        6sc in a mr    m 20
( r2	        [inc] inc inc inc inc inc (12)   
[('inc', 1)] [('inc', 1), ('inc', 2), ('inc', 3), ('inc', 4), ('inc', 5), ('inc', 6)]
window inc  r2	        [inc] inc inc inc inc inc (12)    i 21
window inc  r2	        [inc] inc inc inc inc inc (12)    i 25
( r3	        [sc, inc] sc inc sc inc sc inc sc inc sc inc (18)   
[('sc', 1), ('inc', 2)] [('sc', 1), ('inc', 2), ('sc', 3), ('inc', 4), ('sc', 5), ('inc', 6), ('sc', 7), ('inc', 8), ('sc', 9), ('inc', 10), ('sc', 11), ('inc', 12)]
window inc  r3	        [sc, inc] sc inc sc inc sc inc sc inc sc inc (18)    i 24
window sc i r3	        [sc, inc] sc inc sc inc sc inc sc inc sc inc (18)    s 28
window sc,  r4	        16sc, [slst, ch1, dc, ch1, slst] in flo of one stitch, 1 sc    c 14
window flo  r4	        16sc, [slst, ch1, dc, ch1, slst] in flo of one stitch, 1 sc    f 47
window sc   r4	        16sc, [slst, ch1, dc, ch1, slst] in flo of one stitch, 1sc

In [ ]:
window sc i r1	        6sc in a mr    c 13
window mr   r1	        6sc in a mr    m 20
( r2	        [inc] inc inc inc inc inc (12)   
[('inc', 1)] [('inc', 1), ('inc', 2), ('inc', 3), ('inc', 4), ('inc', 5), ('inc', 6)]
window inc  r2	        [inc] inc inc inc inc inc (12)    i 21
window inc  r2	        [inc] inc inc inc inc inc (12)    i 25
( r3	        [sc, inc] sc inc sc inc sc inc sc inc sc inc (18)   
[('sc', 1), ('inc', 2)] [('sc', 1), ('inc', 2), ('sc', 3), ('inc', 4), ('sc', 5), ('inc', 6), ('sc', 7), ('inc', 8), ('sc', 9), ('inc', 10), ('sc', 11), ('inc', 12)]
window inc  r3	        [sc, inc] sc inc sc inc sc inc sc inc sc inc (18)    i 24
window sc i r3	        [sc, inc] sc inc sc inc sc inc sc inc sc inc (18)    s 28
window sc,  r4	        16sc, [slst, ch1, dc, ch1, slst] in flo of one stitch, 1 sc    c 14
window flo  r4	        16sc, [slst, ch1, dc, ch1, slst] in flo of one stitch, 1 sc    f 47
window sc   r4	        16sc, [slst, ch1, dc, ch1, slst] in flo of one stitch, 1sc    c 68
window sc,  r5	        2sc, [slst, ch1, dc ch1, slst] in  flo of one stitch, 15 sc (the 14th sc should be made in the blo of r3)    c 13
window flo  r5	        2sc, [slst, ch1, dc ch1, slst] in flo of one stitch, 15 sc (the 14th sc should be made in the blo of r3)    l 46
window sc ( r5	        2sc, [slst, ch1, dc ch1, slst] in flo of one stitch, 15sc (the 14th sc should be made in the blo of r3)    c 67
window sc ( r6	        7sc (the third sc should be made in the blo of r4), 4-dcbo , 3 sc,  4-dcbo, 6 sc (18)    c 13
window sc,  r6	        7sc (the third sc should be made in the blo of r4), 4-dcbo , 3sc,  4-dcbo, 6 sc (18)    c 74
window sc ( r6	        7sc (the third sc should be made in the blo of r4), 4-dcbo , 3sc,  4-dcbo, 6sc (18)    c 88
window sc ( r7-9	18sc (18)   3 rounds    c 8
window sc,  r10	        8sc, 4-dcbo, 3 sc, 4-dcbo, 5 sc (18)    c 14
window sc,  r10	        8sc, 4-dcbo, 3sc, 4-dcbo, 5 sc (18)    c 27
window sc ( r10	        8sc, 4-dcbo, 3sc, 4-dcbo, 5sc (18)    c 40